In [1]:
import requests
from labelbox import Client
import requests # to get image from the web
import shutil # to save it locally
import json
import os
import cv2
from imageio import imread
import numpy as np
import time
import sys
from PIL import Image
import math

### Fetch data from labelbox

In [3]:
API_KEY = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9" \
          ".eyJ1c2VySWQiOiJja214ZmY4dGp2OTNxMDc" \
          "1N2s2dzRndzY2Iiwib3JnYW5pemF0aW9uSWQ" \
          "iOiJja214NWg5Znd0ZzUxMDg0ODdpNHZxMDE" \
          "3IiwiYXBpS2V5SWQiOiJja215eG1qNHR5eTh" \
          "sMDc0N2hrdHp3b2ptIiwiaWF0IjoxNjE3Mjg" \
          "0ODIxLCJleHAiOjIyNDg0MzY4MjF9.uJ1wDe" \
          "kRxrYtie31RqpoiM_08tWThlShUZrkqjfCDXI"
client = Client(API_KEY)

project = client.get_project("ckmx5jeqo9u4007902mbiz6k2")
datasets = project.datasets()
exportUrl = project.export_labels()
data = requests.get(exportUrl).json()

### Create dirs

In [4]:
try:
    if not os.path.exists('images'):
        os.makedirs('images')
        print("images dir created!")
    if not os.path.exists('masks'):
        os.makedirs('masks')
        print("masks dir created!")
except OSError:
    print('Error: Creating directory of images')

### Write images and normalized masks

In [6]:
counter = 0
for row in data:
    sys.stdout.write("\r" + "Processing: " + str(counter) + "/" + str(len(data)))
#     if counter == 10:
#         break
    counter += 1
    filename = row['ID'] + '.png'
    image_url = row['Labeled Data']
    try:
        mask_url = row['Label']['objects'][0]['instanceURI']
    except KeyError:
        mask_url = False
        img = Image.new("RGB", (1920, 1080), (0, 0, 0))
        img.save("masks/" + filename, "PNG")

    if mask_url:
        time.sleep(.2)
        mask = imread(mask_url) // 255
        mask = np.delete(mask, np.s_[:3], 2)
        mask = np.reshape(mask, (1080, 1920))
        cv2.imwrite('masks/' + filename, mask)


    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open('images/' + filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        print('\n Image Couldn\'t be retreived')